# Classification on Splice-junction Gene Sequences 


In [1]:
#Imports
%matplotlib inline
import pandas as pd
import numpy as np
from sklearn.model_selection import StratifiedShuffleSplit
from sklearn.preprocessing import LabelEncoder
import tensorflow as tf

MY_MODE_TRAINING = True
MY_MODE_PREDICTION = False
LOGDIR = './TMP/'

### Load the dataset

In [2]:
def load_data(file_path) :
    df = pd.read_csv(file_path, header=None)
    df.columns = ['classlabel', 'name', 'sequence']
    df.tail()
    
    return df

### Pre-process the dataset
* Apply one-hot-encoding to input data
* Take 20% as test data 

In [3]:
def preprocess_data(df) :
    
    # Encoding class labels
    class_le = LabelEncoder()
    y = class_le.fit_transform(df['classlabel'].values)
    #print("y:",y)
    
    # Encoding sequence
    # Here we use one hot encoding to encode the character in DNA sequence. 
    # So each dna sequence is converted to a 60x8 2D array 
    def Seq2Vec(seq):
        s = str(seq).strip()
        CharDict = { "A":[0,0,0,0,0,0,0,1],
                     "G":[0,0,0,0,0,0,1,0],
                     "C":[0,0,0,0,0,1,0,0],
                     "T":[0,0,0,0,1,0,0,0],
                     "D":[0,0,0,1,0,0,0,0],
                     "N":[0,0,1,0,0,0,0,0],
                     "S":[0,1,0,0,0,0,0,0],
                     "R":[1,0,0,0,0,0,0,0]}
        return np.asarray([CharDict[c] for c in s], dtype=np.float32).flatten()

    df['seqvec'] = df['sequence'].apply(Seq2Vec)
    X = np.vstack(df['seqvec'].values)
    print("Total samples:", X.shape[0])
    
    # Split the data set into training/test set
    sss = StratifiedShuffleSplit(n_splits=3, test_size=0.2, random_state=0)
    for train_index, test_index in sss.split(X, y):
        X_train, X_test = X[train_index], X[test_index]
        y_train, y_test = y[train_index], y[test_index]

    print("Training samples: ", X_train.shape[0], "Test samples: ", X_test.shape[0])
    
    return X_train, y_train, X_test, y_test

### DNN model

In [4]:
def model_dnn(X_train, y_train, X_test, y_test, hparam, batch_size=100, n_epochs=2000, 
              use_two_conv=True, n_fc=3, mode_learning=0) :
    
    #DNN approach
    '''
    feature_columns = tf.contrib.learn.infer_real_valued_columns_from_input(X_train)
    dnn_clf = tf.contrib.learn.DNNClassifier(hidden_units=[300, 100], n_classes=3,
                                             feature_columns=feature_columns)
    dnn_clf.fit(x=X_train, y=y_train, batch_size=50, steps=n_epochs)

    from sklearn.metrics import accuracy_score

    y_pred = list(dnn_clf.predict(X_test))
    accuracy = accuracy_score(y_test, y_pred)
    accuracy
    '''
    
    tf.reset_default_graph()
 
    # Hyper parameters
    assert( n_fc>=1 & n_fc<=4 )
    if( n_fc==4 ) : 
        fc1_node = 300
        fc2_node = 100
        fc3_node = 30
    elif( n_fc==3 ) : 
        fc2_node = 300
        fc3_node = 100
    elif( n_fc==2 ) :
        fc3_node = 200
        
    beta = 0.01                  # Regularization 0.01
    dropout_rate = 0.5           # Dropout rate for dropout layer
    learning_rate_set = [0.001, 1e-3, 1e-4, 1e-5]
    assert( mode_learning>=0 & mode_learning<=3 )
    starter_learning_rate = learning_rate_set[mode_learning]
    
    # Variables and inputs
    X = tf.placeholder(tf.float32, shape=(None, 480), name="X")
    input = tf.reshape(X, [-1, 60*8])
    y = tf.placeholder(tf.int32, shape=(None), name="y")
    mode = tf.placeholder(tf.bool, shape=(None), name="mode")
    scale = tf.placeholder(tf.float32, shape=(None), name="scale")
    global_step = tf.placeholder(tf.int32, shape=(None), name="global_step")
    learning_rate = tf.Variable(starter_learning_rate, trainable=False)
    
    if( n_fc == 4 ) :
        # Add the 1st fully connected layer, apply L2 regularizer and xavier weight initializer
        fc1 = tf.layers.dense(
            inputs=input, 
            units=fc1_node, 
            activation=tf.nn.relu, 
            kernel_regularizer=tf.contrib.layers.l2_regularizer(scale=scale, scope=None),
            kernel_initializer=tf.contrib.layers.xavier_initializer(),
            name="fc1")

        #Adding a dropout layer to avoid overfitting
        dropout1 = tf.layers.dropout(inputs=fc1, rate=dropout_rate, training=mode )
     
    if( n_fc>=3 ) : 
        # Add the 2nd fully connected layer
        fc2 = tf.layers.dense(
            inputs= input if n_fc==3 else dropout1, 
            units=fc2_node, 
            activation=tf.nn.relu, 
            kernel_regularizer=tf.contrib.layers.l2_regularizer(scale=scale, scope=None),
            kernel_initializer=tf.contrib.layers.xavier_initializer(),
            name="fc2")
    
        #Adding a dropout layer to avoid overfitting
        dropout2 = tf.layers.dropout(inputs=fc2, rate=dropout_rate, training=mode )
    
    if( n_fc>=2 ) : 
        # Add the 2nd fully connected layer
        fc3 = tf.layers.dense(
            inputs= input if n_fc==2 else dropout2, 
            units=fc3_node, 
            activation=tf.nn.relu, 
            kernel_regularizer=tf.contrib.layers.l2_regularizer(scale=scale, scope=None),
            kernel_initializer=tf.contrib.layers.xavier_initializer(),
            name="fc3")

        #Adding a dropout layer to avoid overfitting
        dropout3 = tf.layers.dropout(inputs=fc3, rate=dropout_rate, training=mode )
    
    # The last output layer
    logits = tf.layers.dense(
        inputs=input if n_fc==1 else dropout3, 
        units=3,
        activation=tf.nn.softmax,
        kernel_regularizer=tf.contrib.layers.l2_regularizer(scale=scale, scope=None),
        kernel_initializer=tf.contrib.layers.xavier_initializer(),
        name="output_FC")
    
    # Cross entropy
    entropy = tf.nn.sparse_softmax_cross_entropy_with_logits(labels=y, logits=logits)
    
    # Apply regularization in training stage and not in prediction stage
    with tf.name_scope('loss'):
        loss = tf.reduce_mean(entropy) + beta*sum( tf.get_collection(tf.GraphKeys.REGULARIZATION_LOSSES) )
    tf.summary.scalar('loss', loss)

    with tf.name_scope('train'):
        if(mode_learning==0):
            # Decayed learning rate
            # Start learning rate as 0.001, decay rate as 0.5, decay 6 steps to ~0.000015 (~1e-5)
            learning_rate = tf.train.exponential_decay(starter_learning_rate, global_step,
                                                       decay_steps=n_epochs//6, decay_rate=0.5, staircase=True)
        tf.summary.scalar('learning_rate', learning_rate)
        # Use Adam optimizer
        optimizer = tf.train.AdamOptimizer(learning_rate=learning_rate).minimize(loss)
    
    with tf.name_scope('accuracy'):
        with tf.name_scope('correct_prediction'):
            correct_prediction = tf.nn.in_top_k(logits, y, 1)
        with tf.name_scope('accuracy'):
            accuracy = tf.reduce_mean(tf.cast(correct_prediction, tf.float32))
    tf.summary.scalar('accuracy', accuracy)

    # Merge all the summaries and write them out to /tmp/mnist_logs (by default)
    merged = tf.summary.merge_all()

    
    init = tf.global_variables_initializer()

    with tf.Session() as sess:
        train_writer = tf.summary.FileWriter(hparam + 'train', sess.graph)
        test_writer = tf.summary.FileWriter(hparam + "test")
        
        init.run()
        n_rounds = X_train.shape[0] // batch_size
        for epoch in range(n_epochs):
            X_batch, y_batch = next_batch(batch_size, X_train, y_train, epoch, n_rounds)
            summary_train, _ = sess.run([merged, optimizer], 
                                        feed_dict= {X: X_batch, y: y_batch, 
                                                    mode: MY_MODE_TRAINING, scale: beta,
                                                    global_step: epoch}) 
            train_writer.add_summary(summary_train, epoch)

            if (epoch % 10 == 0):
                _, acc_train = sess.run([merged, accuracy], feed_dict={X: X_train, y: y_train, 
                                                    mode: MY_MODE_PREDICTION, scale: 0,
                                                    global_step: epoch})
                #train_writer.add_summary(summary_train, epoch)
                
                summary_test, acc_test = sess.run([merged, accuracy], 
                                                  feed_dict={X: X_test, y:y_test, 
                                                            mode: MY_MODE_PREDICTION, scale: 0,
                                                            global_step: epoch})
                test_writer.add_summary(summary_test, epoch)
               
                print(epoch, "Train accuracy:", acc_train,  "Test_accuracy:", acc_test)
    

### Next batch - support function
* Get the next mini_batch from training data

In [5]:
def next_batch(num, data, labels, epoch, rounds):
    ''' 
    Return a total of `num` random samples and labels. 
    Reshuffle the index when running over the total data set epoch%rounds==0
    
    Arg: 
      num, the number of returned data size
      data, the input X
      labels, the label y
      epoch, the current epoch value
      rounds, rounds = label_size // num 

    Returns:
      Return "num" of X and y array
    '''
    global g_idx
    set_cnt = epoch % rounds
    if( (set_cnt) == 0 ) :
        #print("Reshuffling...")
        g_idx = np.arange(0, labels.shape[0])
        np.random.shuffle(g_idx)
        

    idx = g_idx[set_cnt*num:set_cnt*num+num]
    data_shuffle = [data[ i] for i in idx]
    labels_shuffle = [labels[ i] for i in idx]

    return np.asarray(data_shuffle), np.asarray(labels_shuffle)

### CNN Model

In [6]:
def model_cnn(X_train, y_train, X_test, y_test, hparam, batch_size=100, n_epochs=2000, 
              use_two_conv=True, n_fc=3, mode_learning=0) :
    
    #CNN approach
    
    tf.reset_default_graph()
    
    #Hyper parameters
    conv1_depth = 32
    conv2_depth = 64
    conv1_kernel_size = [3, 3]
    conv2_kernel_size = [3, 3]
    dense1_node = 64
    dense2_node = 16
    beta = 0.01                  # Regularization 0.01
    dropout_rate = 0.5           # Dropout rate for dropout layer
    learning_rate_set = [0.01, 1e-3, 1e-4, 1e-5]
    assert( mode_learning>=0 & mode_learning<=3 )
    starter_learning_rate = learning_rate_set[mode_learning]

    X = tf.placeholder(tf.float32, shape=(None, 480), name="X")
    input = tf.reshape(X, [-1, 60, 8, 1])
    y = tf.placeholder(tf.int32, shape=(None), name="y")
    mode = tf.placeholder(tf.bool, shape=(None), name="mode")
    scale = tf.placeholder(tf.float32, shape=(None), name="scale")
    global_step = tf.placeholder(tf.int32, shape=(None), name="global_step")
    learning_rate = tf.Variable(starter_learning_rate, trainable=False)
    
    if( use_two_conv ) :   # Use two conv layer
        # Convolutional Layer #1
        # Computes 32 features using a 3x3 filter.
        # Padding is added to preserve width and height.
        # Input Tensor Shape: [batch_size, 60, 8, 1]
        # Output Tensor Shape: [batch_size, 60, 8, 32]
        conv1 = tf.layers.conv2d(
            inputs=input,
            filters=conv1_depth,
            kernel_size=conv1_kernel_size,
            padding="same",
            activation=tf.nn.relu,
            kernel_regularizer=tf.contrib.layers.l2_regularizer(scale=scale, scope=None),
            kernel_initializer=tf.contrib.layers.xavier_initializer_conv2d(),
            name="conv1")

        #Adding a pooling layer
        pool1 = tf.layers.max_pooling2d(inputs=conv1, pool_size=[2, 2], strides=2)

        # Convolutional Layer #2
        # Computes 64 features using a 3x3 filter.
        # Padding is added to preserve width and height.
        # Input Tensor Shape: [batch_size, 30, 4, 32]
        # Output Tensor Shape: [batch_size, 30, 4, 64]
        conv2 = tf.layers.conv2d(
            inputs=pool1,
            filters=conv2_depth,
            kernel_size=conv2_kernel_size,
            padding="same",
            activation=tf.nn.relu,
            kernel_regularizer=tf.contrib.layers.l2_regularizer(scale=scale, scope=None),
            kernel_initializer=tf.contrib.layers.xavier_initializer_conv2d(),
            name="conv2")

        #Adding a pooling layer
        pool2 = tf.layers.max_pooling2d(inputs=conv2, pool_size=[2, 2], strides=2)
        conv_out_flat = tf.reshape(pool2, [-1, 15 * 2 * conv2_depth])
        
    else :   # Use one conv layer
        conv2 = tf.layers.conv2d(
            inputs=input,
            filters=conv2_depth,
            kernel_size=conv2_kernel_size,
            padding="same",
            activation=tf.nn.relu,
            kernel_regularizer=tf.contrib.layers.l2_regularizer(scale=scale, scope=None),
            kernel_initializer=tf.contrib.layers.xavier_initializer_conv2d(),
            name="conv")

        #Adding a pooling layer
        pool2 = tf.layers.max_pooling2d(inputs=conv2, pool_size=[2, 2], strides=2)
        conv_out_flat = tf.reshape(pool2, [-1, 30 * 4 * conv2_depth])
        
    if( n_fc==3 ) : # Use 3 FC layer
        #Adding a fully connected layer
        dense1 = tf.layers.dense(
            inputs=conv_out_flat, 
            units=dense1_node, 
            activation=tf.nn.relu, 
            kernel_regularizer=tf.contrib.layers.l2_regularizer(scale=scale, scope=None),
            kernel_initializer=tf.contrib.layers.xavier_initializer(),
            name="dense1")

        #Adding a dropout layer to avoid overfitting
        dropout1 = tf.layers.dropout(inputs=dense1, rate=dropout_rate, training=mode )  

        # Add the 2nd fully connected layer
        dense2 = tf.layers.dense(
            inputs=dropout1, 
            units=dense2_node, 
            activation=tf.nn.relu, 
            kernel_regularizer=tf.contrib.layers.l2_regularizer(scale=scale, scope=None),
            kernel_initializer=tf.contrib.layers.xavier_initializer(),
            name="dense2")

        # The last output layer
        logits = tf.layers.dense(
            inputs=dense2, 
            units=3,
            activation=tf.nn.softmax,
            kernel_regularizer=tf.contrib.layers.l2_regularizer(scale=scale, scope=None),
            kernel_initializer=tf.contrib.layers.xavier_initializer(),
            name="output_FC")
        
    elif( n_fc==2 ) : # Use 2 FC layer
        #Adding a fully connected layer
        dense1 = tf.layers.dense(
            inputs=conv_out_flat, 
            units=dense1_node, 
            activation=tf.nn.relu, 
            kernel_regularizer=tf.contrib.layers.l2_regularizer(scale=scale, scope=None),
            kernel_initializer=tf.contrib.layers.xavier_initializer(),
            name="dense1")

        #Adding a dropout layer to avoid overfitting
        dropout1 = tf.layers.dropout(inputs=dense1, rate=dropout_rate, training=mode )  

        # The last output layer
        logits = tf.layers.dense(
            inputs=dropout1, 
            units=3,
            activation=tf.nn.softmax,
            kernel_regularizer=tf.contrib.layers.l2_regularizer(scale=scale, scope=None),
            kernel_initializer=tf.contrib.layers.xavier_initializer(),
            name="output_FC")
        
    else : # Use 1 FC layer
        # The last output layer
        logits = tf.layers.dense(
            inputs=conv_out_flat, 
            units=3,
            activation=tf.nn.softmax,
            kernel_regularizer=tf.contrib.layers.l2_regularizer(scale=scale, scope=None),
            kernel_initializer=tf.contrib.layers.xavier_initializer(),
            name="output_FC")
        
    # Cross entropy
    entropy = tf.nn.sparse_softmax_cross_entropy_with_logits(labels=y, logits=logits)
    
    # Apply regularization in training stage and not in prediction stage
    with tf.name_scope('loss'):
        loss = tf.reduce_mean(entropy) + beta*sum( tf.get_collection(tf.GraphKeys.REGULARIZATION_LOSSES) )
    tf.summary.scalar('loss', loss)

    with tf.name_scope('train'):
        if(mode_learning==0):
            # Decayed learning rate
            # Start learning rate as 0.01, decay rate as 0.5, decay 10 steps to ~0.0000097 (~1e-5)
            learning_rate = tf.train.exponential_decay(starter_learning_rate, global_step,
                                                       decay_steps=n_epochs//10, decay_rate=0.5, staircase=True)
        tf.summary.scalar('learning_rate', learning_rate)
        # Use Adam optimizer
        optimizer = tf.train.AdamOptimizer(learning_rate=learning_rate).minimize(loss)
    
    with tf.name_scope('accuracy'):
        with tf.name_scope('correct_prediction'):
            correct_prediction = tf.nn.in_top_k(logits, y, 1)
        with tf.name_scope('accuracy'):
            accuracy = tf.reduce_mean(tf.cast(correct_prediction, tf.float32))
    tf.summary.scalar('accuracy', accuracy)

    # Merge all the summaries and write them out to /tmp/mnist_logs (by default)
    merged = tf.summary.merge_all()


    
    init = tf.global_variables_initializer()

    with tf.Session() as sess:
        train_writer = tf.summary.FileWriter(hparam + 'train', sess.graph)
        test_writer = tf.summary.FileWriter(hparam + "test")
        
        init.run()
        n_rounds = X_train.shape[0] // batch_size
        for epoch in range(n_epochs):
            X_batch, y_batch = next_batch(batch_size, X_train, y_train, epoch, n_rounds)
            summary_train, _ = sess.run([merged, optimizer], 
                                        feed_dict= {X: X_batch, y: y_batch, 
                                                    mode: MY_MODE_TRAINING, scale: beta,
                                                    global_step: epoch}) 
            train_writer.add_summary(summary_train, epoch)

            if (epoch % 10 == 0):
                _, acc_train = sess.run([merged, accuracy], 
                                        feed_dict={X: X_train, y: y_train, 
                                                   mode: MY_MODE_PREDICTION, scale: 0,
                                                   global_step: epoch})
                #train_writer.add_summary(summary_train, epoch)
                
                summary_test, acc_test = sess.run([merged, accuracy], 
                                                  feed_dict={X: X_test, y:y_test, 
                                                            mode: MY_MODE_PREDICTION, scale: 0,
                                                            global_step: epoch})
                test_writer.add_summary(summary_test, epoch)
               
                print(epoch, "Train accuracy:", acc_train,  "Test_accuracy:", acc_test)
                

### RNN model

In [7]:
def model_rnn(X_train, y_train, X_test, y_test, batch_size=100, n_epochs=2000) :
    
    #RNN approach

    tf.reset_default_graph()

    #Hyper parameters
    n_steps =  60
    n_inputs = 8
    n_neurons = 80
    n_outputs = 3
    beta = 0.2                   # Regularization 0.2
    dropout_rate = 0.5           # Dropout layer for regularization
    learning_rate = 1e-3

    X = tf.placeholder(tf.float32, shape=(None, n_steps * n_inputs), name="X")
    input = tf.reshape(X, [-1, n_steps, n_inputs])
    y = tf.placeholder(tf.int32, shape=(None), name="y")
    mode = tf.placeholder(tf.bool, shape=(None), name="mode")
    scale = tf.placeholder(tf.float32, shape=(None), name="scale")
    
    # RNN layer
    basic_cell = tf.contrib.rnn.BasicRNNCell(num_units=n_neurons)
    outputs, states = tf.nn.dynamic_rnn(basic_cell, input, dtype=tf.float32)
        
    # Dropout layer to avoid overfitting
    dropout = tf.layers.dropout(inputs=states, rate=dropout_rate, training= mode )

    # Fully connected layer - Xavier initializer and L2 regularizer
    logits = tf.contrib.layers.fully_connected(
        inputs=dropout, 
        num_outputs=n_outputs, 
        weights_initializer=tf.contrib.layers.xavier_initializer(),
        weights_regularizer=tf.contrib.layers.l2_regularizer(scale=beta, scope=None),
        activation_fn=tf.nn.softmax 
    )

    entropy = tf.nn.sparse_softmax_cross_entropy_with_logits(labels=y, logits=logits)
    
    # Apply regularization in training stage and not in prediction stage
    with tf.name_scope('loss'):
        loss = tf.reduce_mean(entropy) + beta*sum( tf.get_collection(tf.GraphKeys.REGULARIZATION_LOSSES) )
    tf.summary.scalar('loss', loss)

    # Adam optimizer
    with tf.name_scope('train'):
        optimizer = tf.train.AdamOptimizer(learning_rate=learning_rate).minimize(loss)
    
    with tf.name_scope('accuracy'):
        with tf.name_scope('correct_prediction'):
            correct_prediction = tf.nn.in_top_k(logits, y, 1)
        with tf.name_scope('accuracy'):
            accuracy = tf.reduce_mean(tf.cast(correct_prediction, tf.float32))
    tf.summary.scalar('accuracy', accuracy)

    # Merge all the summaries and write them out to /tmp/mnist_logs (by default)
    merged = tf.summary.merge_all()
    
    
    init = tf.global_variables_initializer()

    with tf.Session() as sess:
        train_writer = tf.summary.FileWriter('./train', sess.graph)
        test_writer = tf.summary.FileWriter('./test')
        
        init.run()
        n_rounds = X_train.shape[0] // batch_size
        #print("n_rounds:", n_rounds)
        
        for epoch in range(n_epochs):
            X_batch, y_batch = next_batch(batch_size, X_train, y_train, epoch, n_rounds)
            
            summary_train, _ = sess.run([merged, optimizer], feed_dict= {X: X_batch, y: y_batch, 
                                                                    mode: MY_MODE_TRAINING, scale: beta }) 
            train_writer.add_summary(summary_train, epoch)

            if (epoch % 10 == 0):
                summary_train, acc_train = sess.run([merged, accuracy], feed_dict={X: X_train, y: y_train, 
                                                    mode: MY_MODE_PREDICTION, scale: 0})
                train_writer.add_summary(summary_train, epoch)
                
                summary_test, acc_test = sess.run([merged, accuracy], feed_dict={X: X_test, y:y_test, 
                                                    mode: MY_MODE_PREDICTION, scale: 0})
                test_writer.add_summary(summary_test, epoch)
               
                print(epoch, "Train accuracy:", acc_train,  "Test_accuracy:", acc_test)
            



### Main 

In [8]:
def make_hparam_string(mode_learning, n_fc, n_conv):
    conv_param = "conv=%d" % (n_conv)
    return LOGDIR+"lr_%d,%s,fc=%d," % (mode_learning, conv_param, n_fc)

def main(unused_argv):
    
    # Load data
    df = load_data("..\data\splice.data")
    
    # Preprocess data and split training and validation data
    X_train, y_train, X_test, y_test = preprocess_data(df)
    
    # DNN model
    for mode_learning in [0, 1, 2, 3]:
        for n_fc in [4, 3, 2, 1]:
            hparam = make_hparam_string(mode_learning=mode_learning, n_fc=n_fc, n_conv=0)
            model_dnn(X_train, y_train, X_test, y_test, hparam=hparam, batch_size=100, n_epochs=5000, 
                      n_fc=n_fc, mode_learning=mode_learning)
        
    # CNN model
    #model_cnn(X_train, y_train, X_test, y_test, batch_size=100, n_epochs=2000, 
    #                  use_two_conv=True, n_fc=3, mode_learning=2)
    
    #Select 
    '''
    # Search hyper parameters over the different # of layers
    for n_fc in [3, 2, 1]:
        for n_conv in [2, 1]:
            # Construct a hyperparameter string for each one (example: "lr_1E-3,fc=2,conv=2)
            hparam = make_hparam_string(2, n_fc, n_conv=n_conv)

            print('Starting run for %s' % hparam)
            model_cnn(X_train, y_train, X_test, y_test, batch_size=100, n_epochs=3000, hparam=hparam,
                      use_two_conv=(n_conv==2), n_fc=n_fc, mode_learning=2)
    '''
    '''
    # Search and compare the best learning rate
    for mode_learning in [0, 1, 2, 3]:
        n_fc=3
        for two_conv in [1]:
            # Construct a hyperparameter string for each one (example: "lr_1E-3,fc=2,conv=2)
            hparam = make_hparam_string(mode_learning=mode_learning, n_fc=n_fc, n_conv=n_conv)

            print('Starting run for %s' % hparam)
            model_cnn(X_train, y_train, X_test, y_test, batch_size=100, n_epochs=3000, hparam=hparam,
                      use_two_conv=(n_conv==2), n_fc=n_fc, mode_learning=mode_learning)
    '''
    
    # RNN model
    #model_rnn(X_train=X_train, y_train=y_train, X_test=X_test, y_test=y_test, batch_size=100, n_epochs=3000)
    

In [9]:
if __name__ == "__main__":
  tf.app.run()

Total samples: 3190
Training samples:  2552 Test samples:  638
0 Train accuracy: 0.386364 Test_accuracy: 0.382445
10 Train accuracy: 0.518809 Test_accuracy: 0.518809
20 Train accuracy: 0.518809 Test_accuracy: 0.518809
30 Train accuracy: 0.518809 Test_accuracy: 0.518809
40 Train accuracy: 0.596787 Test_accuracy: 0.576802
50 Train accuracy: 0.847179 Test_accuracy: 0.840125
60 Train accuracy: 0.913009 Test_accuracy: 0.913793
70 Train accuracy: 0.933777 Test_accuracy: 0.916928
80 Train accuracy: 0.952586 Test_accuracy: 0.931035
90 Train accuracy: 0.962774 Test_accuracy: 0.948276
100 Train accuracy: 0.961207 Test_accuracy: 0.946708
110 Train accuracy: 0.965909 Test_accuracy: 0.952978
120 Train accuracy: 0.968652 Test_accuracy: 0.956113
130 Train accuracy: 0.968652 Test_accuracy: 0.956113
140 Train accuracy: 0.971003 Test_accuracy: 0.956113
150 Train accuracy: 0.968652 Test_accuracy: 0.948276
160 Train accuracy: 0.972571 Test_accuracy: 0.954545
170 Train accuracy: 0.972962 Test_accuracy: 0.9

1540 Train accuracy: 0.993339 Test_accuracy: 0.952978
1550 Train accuracy: 0.993339 Test_accuracy: 0.952978
1560 Train accuracy: 0.993339 Test_accuracy: 0.952978
1570 Train accuracy: 0.993339 Test_accuracy: 0.949843
1580 Train accuracy: 0.993339 Test_accuracy: 0.949843
1590 Train accuracy: 0.993339 Test_accuracy: 0.949843
1600 Train accuracy: 0.993339 Test_accuracy: 0.952978
1610 Train accuracy: 0.993339 Test_accuracy: 0.956113
1620 Train accuracy: 0.993339 Test_accuracy: 0.954545
1630 Train accuracy: 0.993339 Test_accuracy: 0.952978
1640 Train accuracy: 0.993339 Test_accuracy: 0.952978
1650 Train accuracy: 0.993339 Test_accuracy: 0.952978
1660 Train accuracy: 0.993339 Test_accuracy: 0.951411
1670 Train accuracy: 0.993339 Test_accuracy: 0.951411
1680 Train accuracy: 0.993339 Test_accuracy: 0.951411
1690 Train accuracy: 0.993339 Test_accuracy: 0.954545
1700 Train accuracy: 0.993339 Test_accuracy: 0.954545
1710 Train accuracy: 0.993339 Test_accuracy: 0.954545
1720 Train accuracy: 0.99333

3090 Train accuracy: 0.99373 Test_accuracy: 0.949843
3100 Train accuracy: 0.99373 Test_accuracy: 0.949843
3110 Train accuracy: 0.99373 Test_accuracy: 0.948276
3120 Train accuracy: 0.99373 Test_accuracy: 0.948276
3130 Train accuracy: 0.99373 Test_accuracy: 0.948276
3140 Train accuracy: 0.99373 Test_accuracy: 0.948276
3150 Train accuracy: 0.99373 Test_accuracy: 0.948276
3160 Train accuracy: 0.99373 Test_accuracy: 0.949843
3170 Train accuracy: 0.99373 Test_accuracy: 0.951411
3180 Train accuracy: 0.99373 Test_accuracy: 0.951411
3190 Train accuracy: 0.99373 Test_accuracy: 0.952978
3200 Train accuracy: 0.99373 Test_accuracy: 0.952978
3210 Train accuracy: 0.99373 Test_accuracy: 0.952978
3220 Train accuracy: 0.99373 Test_accuracy: 0.951411
3230 Train accuracy: 0.99373 Test_accuracy: 0.952978
3240 Train accuracy: 0.99373 Test_accuracy: 0.952978
3250 Train accuracy: 0.99373 Test_accuracy: 0.952978
3260 Train accuracy: 0.99373 Test_accuracy: 0.949843
3270 Train accuracy: 0.99373 Test_accuracy: 0.

4640 Train accuracy: 0.99373 Test_accuracy: 0.951411
4650 Train accuracy: 0.99373 Test_accuracy: 0.951411
4660 Train accuracy: 0.99373 Test_accuracy: 0.951411
4670 Train accuracy: 0.99373 Test_accuracy: 0.951411
4680 Train accuracy: 0.99373 Test_accuracy: 0.951411
4690 Train accuracy: 0.99373 Test_accuracy: 0.951411
4700 Train accuracy: 0.99373 Test_accuracy: 0.951411
4710 Train accuracy: 0.99373 Test_accuracy: 0.951411
4720 Train accuracy: 0.99373 Test_accuracy: 0.951411
4730 Train accuracy: 0.99373 Test_accuracy: 0.951411
4740 Train accuracy: 0.99373 Test_accuracy: 0.951411
4750 Train accuracy: 0.99373 Test_accuracy: 0.951411
4760 Train accuracy: 0.99373 Test_accuracy: 0.951411
4770 Train accuracy: 0.99373 Test_accuracy: 0.951411
4780 Train accuracy: 0.99373 Test_accuracy: 0.951411
4790 Train accuracy: 0.99373 Test_accuracy: 0.951411
4800 Train accuracy: 0.99373 Test_accuracy: 0.951411
4810 Train accuracy: 0.99373 Test_accuracy: 0.951411
4820 Train accuracy: 0.99373 Test_accuracy: 0.

1200 Train accuracy: 0.994122 Test_accuracy: 0.95768
1210 Train accuracy: 0.994122 Test_accuracy: 0.95768
1220 Train accuracy: 0.994122 Test_accuracy: 0.95768
1230 Train accuracy: 0.994122 Test_accuracy: 0.956113
1240 Train accuracy: 0.994122 Test_accuracy: 0.956113
1250 Train accuracy: 0.994122 Test_accuracy: 0.949843
1260 Train accuracy: 0.994122 Test_accuracy: 0.954545
1270 Train accuracy: 0.994122 Test_accuracy: 0.956113
1280 Train accuracy: 0.994122 Test_accuracy: 0.956113
1290 Train accuracy: 0.994122 Test_accuracy: 0.954545
1300 Train accuracy: 0.994122 Test_accuracy: 0.954545
1310 Train accuracy: 0.994122 Test_accuracy: 0.949843
1320 Train accuracy: 0.994122 Test_accuracy: 0.951411
1330 Train accuracy: 0.994122 Test_accuracy: 0.951411
1340 Train accuracy: 0.994122 Test_accuracy: 0.952978
1350 Train accuracy: 0.994122 Test_accuracy: 0.95768
1360 Train accuracy: 0.994122 Test_accuracy: 0.951411
1370 Train accuracy: 0.994122 Test_accuracy: 0.952978
1380 Train accuracy: 0.994122 Te

...

920 Train accuracy: 0.496473 Test_accuracy: 0.503135
930 Train accuracy: 0.496865 Test_accuracy: 0.504702
940 Train accuracy: 0.497649 Test_accuracy: 0.504702
950 Train accuracy: 0.498041 Test_accuracy: 0.504702
960 Train accuracy: 0.499216 Test_accuracy: 0.504702
970 Train accuracy: 0.5 Test_accuracy: 0.50627
980 Train accuracy: 0.501567 Test_accuracy: 0.509404
990 Train accuracy: 0.501567 Test_accuracy: 0.509404
1000 Train accuracy: 0.501567 Test_accuracy: 0.509404
1010 Train accuracy: 0.501959 Test_accuracy: 0.509404
1020 Train accuracy: 0.502743 Test_accuracy: 0.509404
1030 Train accuracy: 0.503527 Test_accuracy: 0.507837
1040 Train accuracy: 0.50431 Test_accuracy: 0.509404
1050 Train accuracy: 0.50431 Test_accuracy: 0.510972
1060 Train accuracy: 0.504702 Test_accuracy: 0.510972
1070 Train accuracy: 0.505486 Test_accuracy: 0.510972
1080 Train accuracy: 0.505878 Test_accuracy: 0.510972
1090 Train accuracy: 0.505878 Test_accuracy: 0.509404
1100 Train accuracy: 0.505878 Test_accuracy:

2460 Train accuracy: 0.519984 Test_accuracy: 0.518809
2470 Train accuracy: 0.519984 Test_accuracy: 0.518809
2480 Train accuracy: 0.519984 Test_accuracy: 0.518809
2490 Train accuracy: 0.519984 Test_accuracy: 0.518809
2500 Train accuracy: 0.519984 Test_accuracy: 0.518809
2510 Train accuracy: 0.519984 Test_accuracy: 0.518809
2520 Train accuracy: 0.519984 Test_accuracy: 0.518809
2530 Train accuracy: 0.519984 Test_accuracy: 0.518809
2540 Train accuracy: 0.519984 Test_accuracy: 0.518809
2550 Train accuracy: 0.519984 Test_accuracy: 0.518809
2560 Train accuracy: 0.519984 Test_accuracy: 0.518809
2570 Train accuracy: 0.519984 Test_accuracy: 0.518809
2580 Train accuracy: 0.519984 Test_accuracy: 0.518809
2590 Train accuracy: 0.519984 Test_accuracy: 0.518809
2600 Train accuracy: 0.519984 Test_accuracy: 0.518809
2610 Train accuracy: 0.519984 Test_accuracy: 0.518809
2620 Train accuracy: 0.519984 Test_accuracy: 0.518809
2630 Train accuracy: 0.519984 Test_accuracy: 0.518809
2640 Train accuracy: 0.51998

4000 Train accuracy: 0.521552 Test_accuracy: 0.523511
4010 Train accuracy: 0.522335 Test_accuracy: 0.525078
4020 Train accuracy: 0.522727 Test_accuracy: 0.525078
4030 Train accuracy: 0.522727 Test_accuracy: 0.525078
4040 Train accuracy: 0.522727 Test_accuracy: 0.525078
4050 Train accuracy: 0.522727 Test_accuracy: 0.525078
4060 Train accuracy: 0.522727 Test_accuracy: 0.525078
4070 Train accuracy: 0.522727 Test_accuracy: 0.525078
4080 Train accuracy: 0.522727 Test_accuracy: 0.525078
4090 Train accuracy: 0.522727 Test_accuracy: 0.525078
4100 Train accuracy: 0.522727 Test_accuracy: 0.525078
4110 Train accuracy: 0.522727 Test_accuracy: 0.525078
4120 Train accuracy: 0.522727 Test_accuracy: 0.525078
4130 Train accuracy: 0.523119 Test_accuracy: 0.526646
4140 Train accuracy: 0.523119 Test_accuracy: 0.526646
4150 Train accuracy: 0.523119 Test_accuracy: 0.526646
4160 Train accuracy: 0.523119 Test_accuracy: 0.526646
4170 Train accuracy: 0.523119 Test_accuracy: 0.529781
4180 Train accuracy: 0.52351

SystemExit: 

C:\Users\qli45\AppData\Local\Continuum\Anaconda3\lib\site-packages\IPython\core\interactiveshell.py:2889: UserWarning: To exit: use 'exit', 'quit', or Ctrl-D.
  warn("To exit: use 'exit', 'quit', or Ctrl-D.", stacklevel=1)
